# Chapter 10: Model Context Protocol (MCP)

Key Takeaways:
- **Model Context Protocol (MCP)** is a standard for connecting AI models to external data and tools.
- **Filesystem MCP**: Allows agents to securely interact with the local filesystem.
- **FastMCP**: A Python library for quickly building custom MCP servers.
- **Client Integration**: How to connect an `LlmAgent` to both stdio and HTTP-based MCP servers.

### Heuristic: *Standardize tool access via MCP.*

## Setup and Initialization

In [ ]:
import os
import nest_asyncio
from dotenv import load_dotenv
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioServerParameters, SseConnectionParams
from google.genai.types import Content, Part

# Allow nested event loops
nest_asyncio.apply()

load_dotenv()

# --- Configuration ---
PROJECT_ROOT = os.path.dirname(os.getcwd())
SCRIPTS_DIR = os.path.join(PROJECT_ROOT, "scripts")

print(f"✅ Configuration Loaded:")
print(f"   Project Root: {PROJECT_ROOT}")
print(f"   Scripts Directory: {SCRIPTS_DIR}")

## 1. Using the Filesystem MCP Server

We can use the standard `@modelcontextprotocol/server-filesystem` to give an agent controlled access to a specific directory.

In [ ]:
# Create a safe managed directory for the agent
TARGET_FOLDER_PATH = os.path.join(os.getcwd(), "../scripts/mcp_managed_files")
os.makedirs(TARGET_FOLDER_PATH, exist_ok=True)

print(f"📂 Agent Working Directory: {TARGET_FOLDER_PATH}")

In [ ]:
# Configure the Agent with the MCP Toolset
fs_agent = LlmAgent(
    model='gemini-2.5-flash',
    name='filesystem_assistant',
    instruction=(
        'Help the user manage their files. '
        'You can list, read, and write files in your working directory.'
        f'You are operating in: {TARGET_FOLDER_PATH}'
    ),
    tools=[
        MCPToolset(
            connection_params=StdioServerParameters(
                command='npx',
                args=[
                    "-y",
                    "@modelcontextprotocol/server-filesystem",
                    TARGET_FOLDER_PATH, # The allowed directory
                ],
            )
        )
    ],
)

print("🤖 Filesystem Agent Created")

In [ ]:
# Setup session service and runner
session_service = InMemorySessionService()
app_name, user_id, session_id = "mcp_app", "user1", "session1"

session = await session_service.create_session(
    app_name=app_name, user_id=user_id, session_id=session_id
)

runner = Runner(
    agent=fs_agent,
    app_name=app_name,
    session_service=session_service
)

# Ask the agent to create and list a file
user_message = Content(parts=[Part(text=
    "Create a file named 'hello_mcp.txt' with the content 'Hello from the Model Context Protocol!' "
    "and then list the files in the directory."
)])

for event in runner.run(user_id=user_id, session_id=session_id, new_message=user_message):
    if event.is_final_response():
        print(event.content.parts[0].text)

## 2. Custom MCP Server with FastMCP

You can create your own MCP servers using Python and `fastmcp`. We have created a simple server script in `scripts/fastmcp_server.py`.

In [ ]:
server_script_path = os.path.join(SCRIPTS_DIR, "fastmcp_server.py")

with open(server_script_path, 'r') as f:
    print(f.read())

### Running the Server

To use this server, it needs to be running. In a production environment, this would be a persistent service.

**Action Required**: Open a terminal and run the following command to start the server on port 8000:

```bash
python3 ../scripts/fastmcp_server.py
```

*Note: Ensure you have installed the requirements (`pip install -r requirements.txt`) which includes `fastmcp`.*

## 3. Connecting to the Custom Server

Once the server is running locally on port 8000, we can connect an agent to it using `SseConnectionParams` (since our script uses SSE over HTTP).

In [ ]:
FASTMCP_SERVER_URL = "http://127.0.0.1:8000/sse"

greeter_agent = LlmAgent(
    model='gemini-2.5-flash',
    name='greeter_agent',
    instruction='You are a friendly assistant. Use the "greet" tool to say hello.',
    tools=[
        MCPToolset(
            connection_params=SseConnectionParams(
                url=FASTMCP_SERVER_URL,
            ),
            tool_filter=['greet'] # Optional: whitelist specific tools
        )
    ],
)

print("🤖 Greeter Agent Created")

> **Note**: The cell below will fail if the `fastmcp_server.py` is not running in a separate terminal.

In [ ]:
try:
    # Setup session service and runner for the greeter agent
    greeter_session_service = InMemorySessionService()
    greeter_app_name, greeter_user_id, greeter_session_id = "greeter_app", "user1", "session1"

    session = await greeter_session_service.create_session(
        app_name=greeter_app_name, user_id=greeter_user_id, session_id=greeter_session_id
    )

    greeter_runner = Runner(
        agent=greeter_agent,
        app_name=greeter_app_name,
        session_service=greeter_session_service
    )

    user_message = Content(parts=[Part(text="Greet my friend Alice.")])
    for event in greeter_runner.run(user_id=greeter_user_id, session_id=greeter_session_id, new_message=user_message):
        if event.is_final_response():
            print(event.content.parts[0].text)
except Exception as e:
    print(f"❌ Connection Failed: {e}")
    print("Did you remember to start the fastmcp_server.py in a terminal?")

## Conclusion

MCP provides a unified way to equip agents with tools, whether they are standard system utilities like the filesystem or custom business logic exposed via FastMCP.